# Calculate zonal nighttime lights and animated nighttime lights

This notebook calculates zonal statistics on VIIRS nighttime lights for a series of industrial parks. For each of those parks, we calculate a series of buffers, and summarize nighttime lights.

Finally, and animation of nighttime lights is created for a 10km buffer around the IP.

In [1]:
import sys, os, importlib
import rasterio, boto3

import pandas as pd
import geopandas as gpd

from shapely.geometry import Point, box

sys.path.append("../../../../gostrocks/src")

import GOSTRocks.rasterMisc as rMisc
import GOSTRocks 
from GOSTRocks.misc import tPrint


/home/wb411133/.local/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [2]:
ip_locations = "Data/IP_Locations.csv"
out_folder = "/home/wb411133/temp/ETH_NTL/" 
viirs_folder = os.path.join(out_folder, "Data/VIIRS")
zonal_res = os.path.join(out_folder, "Data/IP_Locations_NTL_ZONAL.csv")

if not os.path.exists(viirs_folder):
    os.makedirs(viirs_folder)
    
# Read in csv, re project to UTM, and write to file
inD = pd.read_csv(ip_locations)
inD_geom = [Point(x['Lon'], x['Lat']) for idx, x in inD.iterrows()]
inD = gpd.GeoDataFrame(inD, geometry=inD_geom, crs="epsg:4326")
inD = inD.to_crs("EPSG:20138")
inD.to_file(ip_locations.replace(".csv", ".geojson"), driver="GeoJSON")

In [3]:
inD['Industrial Park'].unique()

array(['Adama IP', 'Bole Lemi IP', 'Debre Birhan', 'Dire Dawa',
       'Hawassa IP', 'ICT IP', 'Kilinto Pharmaceutical IP',
       'Kombolcha IP', 'Mekele IP', 'Bahir Dar IP', 'CCCC Arerti', 'DBL',
       'Eastern Industrial Zone', 'Huajian IP', 'Jimma IP',
       'Mojo George Shoe Industrial Zone', 'Vogue/Velocity'], dtype=object)

# Zonal Stats

In [4]:
buffers = list(range(5, 51, 5))
buffers.append(2)
buffers.append(3)

try:
    del final
except:
    pass

for buf in buffers:    
    curD = inD.copy()
    curD['geometry'] = curD.buffer(buf * 1000)
    curD['BUFFER_KM'] = buf
    try:
        final = final.append(curD)
    except:
        final = curD
final = final.sort_values(["Industrial Park", "BUFFER_KM"])
inD = final
inD = inD.to_crs("epsg:4326")
inD.to_file(ip_locations.replace(".csv", "_BUFFERED.geojson"), driver="GeoJSON")

In [5]:
def get_all_s3_keys(s3, bucket, prefix=''):
    """Get a list of all keys in an S3 bucket."""
    keys = []

    kwargs = {'Bucket': bucket, 'Prefix': prefix}
    while True:
        resp = s3.list_objects_v2(**kwargs)
        for obj in resp['Contents']:
            keys.append(obj['Key'])
        try:
            kwargs['ContinuationToken'] = resp['NextContinuationToken']
        except KeyError:
            break
    return keys

# Get a list of the VIIRS images in S3. This example leverages the GOST teams S3 bucket
s3_base = 'wbgdecinternal-ntl'
s3 = boto3.client('s3')

all_objects = get_all_s3_keys(s3, s3_base, "NTL/VIIRS_UNZIP")

In [6]:
sel_images = []
for obj in all_objects:
    s3_path = f's3://{s3_base}/{obj}'
    try:
        curR = rasterio.open(s3_path)
        cur_extent = box(*curR.bounds)
        col_name = obj.split("/")[-1].replace(".tif", '')
        if cur_extent.intersects(inD.unary_union) and (not col_name in inD.columns) \
            and (not "cf_cvg" in obj) and (not "n_cf" in obj) :        
            tPrint(obj)
            res = rMisc.zonalStats(inD, curR, minVal=0.05)
            res = pd.DataFrame(res,columns=['SUM','MIN','MAX','MEAN'])
            inD[col_name] = res['SUM'].values
    except:
        tPrint("Error Processing %s" % obj)


10:02:09	NTL/VIIRS_UNZIP/viirs_201204/TILE2/SVDNB_npp_20120401-20120430_75N060W_vcmcfg_v10_c201605121456.avg_rade9h.tif
10:02:24	NTL/VIIRS_UNZIP/viirs_201205/TILE2/SVDNB_npp_20120501-20120531_75N060W_vcmcfg_v10_c201605121458.avg_rade9h.tif
10:02:40	NTL/VIIRS_UNZIP/viirs_201206/TILE2/SVDNB_npp_20120601-20120630_75N060W_vcmcfg_v10_c201605121459.avg_rade9h.tif
10:02:54	NTL/VIIRS_UNZIP/viirs_201207/TILE2/SVDNB_npp_20120701-20120731_75N060W_vcmcfg_v10_c201605121509.avg_rade9.tif
10:03:10	NTL/VIIRS_UNZIP/viirs_201208/TILE2/SVDNB_npp_20120801-20120831_75N060W_vcmcfg_v10_c201602121348.avg_rade9.tif
10:03:27	NTL/VIIRS_UNZIP/viirs_201210/TILE2/SVDNB_npp_20121001-20121031_75N060W_vcmcfg_v10_c201602051401.avg_rade9.tif
10:03:42	NTL/VIIRS_UNZIP/viirs_201211/TILE2/SVDNB_npp_20121101-20121130_75N060W_vcmcfg_v10_c201601270845.avg_rade9.tif
10:03:58	NTL/VIIRS_UNZIP/viirs_201212/TILE2/SVDNB_npp_20121201-20121231_75N060W_vcmcfg_v10_c201601041440.avg_rade9.tif
10:04:09	Error Processing NTL/VIIRS_UNZIP/vii

10:19:25	NTL/VIIRS_UNZIP/viirs_201711/TILE2/SVDNB_npp_20171101-20171130_75N060W_vcmcfg_v10_c201712040930.avg_rade9h.tif
10:19:42	NTL/VIIRS_UNZIP/viirs_201712/TILE2/SVDNB_npp_20171201-20171231_75N060W_vcmcfg_v10_c201801021747.avg_rade9h.tif
10:19:58	NTL/VIIRS_UNZIP/viirs_201801/TILE2/SVDNB_npp_20180101-20180131_75N060W_vcmcfg_v10_c201805221252.avg_rade9h.tif
10:20:13	NTL/VIIRS_UNZIP/viirs_201802/TILE2/SVDNB_npp_20180201-20180228_75N060W_vcmcfg_v10_c201803012000.avg_rade9h.tif
10:20:30	NTL/VIIRS_UNZIP/viirs_201803/TILE2/SVDNB_npp_20180301-20180331_75N060W_vcmcfg_v10_c201804022005.avg_rade9h.tif
10:20:45	NTL/VIIRS_UNZIP/viirs_201804/TILE2/SVDNB_npp_20180401-20180430_75N060W_vcmcfg_v10_c201805021400.avg_rade9h.tif
10:21:01	NTL/VIIRS_UNZIP/viirs_201805/TILE2/SVDNB_npp_20180501-20180531_75N060W_vcmcfg_v10_c201806061100.avg_rade9h.tif
10:21:16	NTL/VIIRS_UNZIP/viirs_201807/TILE2/SVDNB_npp_20180701-20180731_75N060W_vcmcfg_v10_c201812111300.avg_rade9h.tif
10:21:31	NTL/VIIRS_UNZIP/viirs_201808/TI

In [7]:
origD = inD.copy()

In [8]:
# drop columns that contain counts ("n_cf")
inD = inD.loc[:,[not 'n_cf' in x for x in inD.columns]]

# rename
def fix_col_names(x):
    if 'DNB' in x:
        return(x.split("_")[2].split("-")[0])
    else:
        return(x)
    
new_cols = [fix_col_names(x) for x in inD.columns]
inD.columns = new_cols

In [9]:
pd.DataFrame(inD.drop(['geometry'], axis=1)).to_csv(os.path.join(out_folder, "NTL_2020_summaries.csv"))

# Generate animated maps

In [ ]:
# If you want to clip out the raster data for the below mapping, run this block
sel_D = inD.loc[inD['BUFFER_KM'] == 10]
for idx, row in sel_D.iterrows():
    tPrint(out_folder)
    
    out_folder = os.path.join(viirs_folder, row['Industrial Park'])
    out_map_folder = os.path.join(out_folder, "MAPS")
    if not os.path.exists(out_map_folder):
        os.makedirs(out_map_folder)
    
    viirs_files = []
    for cur_tif in all_files:
        file = f'{cur_tif.split("/")[5]}.tif'
        out_file = os.path.join(os.path.join(out_folder, "%s" % file))
        viirs_files.append(out_file)
        if not os.path.exists(out_file):
            row['geometry'] = box(*row.geometry.bounds)
            rMisc.clipRaster(rasterio.open(cur_tif), gpd.GeoDataFrame(pd.DataFrame(sel_D.loc[idx]).transpose(), geometry="geometry", crs=inD.crs), out_file)    
            
    viirs_files.sort()
    out_files = []
    for f in viirs_files:
        out_file = os.path.join(out_map_folder, os.path.basename(f))
        out_files.append(out_file)
        rMisc.map_viirs(f, out_file, text_x = 0, text_y = 6)
        
    kwargs = {'duration':0.3}
    images=[]
    for filename in out_files:
        images.append(imageio.imread(filename))

    imageio.mimsave("%s_timelapse.gif" % out_folder, images, **kwargs)
    

In [ ]:
importlib.reload(rMisc)
in_tif = f
rMisc.map_viirs(f, out_file, text_x = 0, text_y = 6)

In [ ]:
rasterio.open(in_tif).read()

In [ ]:
importlib.reload(rMisc)
for idx, row in sel_D.iterrows():
    out_folder = os.path.join(viirs_folder, row['Industrial Park'])
    viirs_files = [os.path.join(out_folder, x) for x in os.listdir(out_folder) if x.endswith("tif")]
    

In [ ]:
import imageio
for filename in out_files:
    images.append(imageio.imread(filename))

imageio.mimsave("%s_timelapse.gif" % out_folder, images, **kwargs)

In [ ]:
"%s_timelapse.gif" % out_folder